In [1]:
import ipywidgets.widgets as widgets
from traitlets import Unicode, Dict
from random import randint
from itertools import count
from shapely.geometry import Polygon, MultiPolygon

In [2]:
shapes = [ [[0,0],[0,1], [1,1], [1,0]], [[1,0],[1,1], [2,1], [2,0]], [[2,0],[2,1], [3,1], [3,0]],]
mp = MultiPolygon([Polygon(s) for s in shapes])
svg = mp._repr_svg_()

In [3]:
class PolygonSelector(widgets.DOMWidget):
    _view_name = Unicode('PolygonSelectorView').tag(sync=True)
    _view_module = Unicode('polygonselector').tag(sync=True)
    groups_dict = Dict().tag(sync=True)
    content = Unicode().tag(sync=True)

    html_template = '''
    <style>
    # polygonGeometry .selectedPolygon {{
        fill: {fill_selected!r};
    }}
    # polygonGeometry .hoveredPolygon {{
        fill: {fill_hovered!r};
    }}
    {selection_styles}
    </style>
    <button id = "clearBtn"> Clear </button>
    <input placeholder = "Name this collection" id = "name" />
    <button id = "saveBtn"> Save </button>
    <div id = "polygonGeometry">{svg}</div>
    '''

    # provide some default colors; can override if desired
    fill_selected = "plum"
    fill_hovered = "lavender"
    group_colors = ["#{:06X}".format(randint(0,0xFFFFFF)) for _ in range(100)]

    def __init__(self, svg):
        super().__init__()
        self.update_content(svg)

    def update_content(self, svg):
        self.content = self.html_template.format(
            fill_selected = self.fill_selected,
            fill_hovered = self.fill_hovered,
            selection_styles = self.selection_styles,
            svg = svg
        )

    @property
    def selection_styles(self):
        return "".join(f'''
        # polygonGeometry .selection_{group_idx} {{
            fill: {self.group_colors[group_idx]!r};
        }}
        ''' for group_idx in range(len(self.groups_dict)))


In [61]:
%%javascript
require.undef('polygonselector');

define('polygonselector', ["@jupyter-widgets/base"], function(widgets) {

    var PolygonSelectorView = widgets.DOMWidgetView.extend({

        initialized: 0,

        init_render: function(){
            
            // Each path element is a polygon
            const polygons = this.el.querySelectorAll('#polygonGeometry path');

            // Add click event to polygons
            Array(polygons).forEach(function(path, i) {
              path.addEventListener('click', function() {
                let collection = this.collection
                if (collection.selections.includes(i)) {
                  path.classList.remove('selectedPolygon')
                  this.remove(i);
                  console.log('path #', i, ' removed');
                } else {
                  path.classList.add('selectedPolygon')
                  this.add(i);
                  console.log('path #', i, ' added');
                }
              })
              console.log('path #', i, ' click set');

            // Attach functions to buttons
            this.el.getElementById('#clearBtn').addEventListener('click', clear);
            this.console.log('clearBtn action set');
            this.el.getElementById('#saveBtn').addEventListener('click', save);
            this.console.log('saveBtn action set');

            }, this);
   
        },


        // Add item to selection list
        add: function(id) {
          let collection = this.collection
          collection.selections.push(id);
          console.log('pushed #', id);
        },

        // Remove item from selection list
        remove: function(id) {
          let collection = this.collection
          collection.selections = collection.selections.filter(function(_id) {
            return _id !== id;
          })
          console.log('filtered #', id);
        },

        // Remove all items
        clear: function() {
          console.log('clear() clicked');
          let collection = this.collection
          collection.selections = [];
          Array.from(document.querySelectorAll('.selectedPolygon')).forEach(function(path) {
            path.classList.remove('selectedPolygon');
          })
          console.log('Data cleared');
        },

        // Send current selection back up-wire
        function save() {
          console.log('save() clicked');
          let collection = this.collection
          const nameInput = element.find('#name');
          if (!nameInput.val() || collection.selections.length < 1) {
            alert('Collections must have a name and selected polygons');
          }
          else {
            collection.name = nameInput.val();
            console.log('You saved some data');
            console.log("Selection Name: ", collection.name);
            console.log(collection.selections);
          }
        },

        render: function() {
            if (this.initialized===0) {
                this.initialized = 1;
                this.init_render();
                };
            this.content_changed();
            this.model.on('change:content', this.content_changed, this);
        },

        content_changed: function() {
            this.el.innerHTML = `${this.model.get('content')}`;
        },
    });

    return {
        PolygonSelectorView : PolygonSelectorView
    };
});

<IPython.core.display.Javascript object>

In [59]:
PolygonSelector(svg)

PolygonSelector(content='\n    <style>\n    # polygonGeometry .selectedPolygon {\n        fill: \'plum\';\n   …

In [7]:
%%javascript

element.html(`
<style>
  #polygonGeometry path.selectedPolygon {
    fill: blue;
  }''' f'''
</style>
<button id="clearBtn">Clear</button>
<input placeholder="Name this collection" id="name" />
<button id="saveBtn">Save</button>
<div id="polygonGeometry">{svg}</div>
`)''' '''

// Mutable collection data to send back up the wire "somehow"
let collection = {
  name: '',
  selections: []
}

// Each path element is a polygon
const polygons = element.find('#polygonGeometry path');

// Add click event to polygons
polygons.each(function(i, path) {
  path.addEventListener('click', function() {
    if (collection.selections.includes(i)) {
      path.classList.remove('selectedPolygon')
      remove(i);
      console.log('path #', i, ' removed');
    } else {
      path.classList.add('selectedPolygon')
      add(i);
      console.log('path #', i, ' added');
    }
  })
  console.log('path #', i, ' click set');
})

// Attach functions to buttons
element.find('#clearBtn').click(clear);
console.log('clearBtn action set');
element.find('#saveBtn').click(save);
console.log('saveBtn action set');

// Add item to selection list
function add(id) {
  collection.selections.push(id);
  console.log('pushed #', id);
}

// Remove item from selection list
function remove(id) {
  collection.selections = collection.selections.filter(function(_id) {
    return _id !== id;
  })
  console.log('filtered #', id);
}

// Remove all items
function clear() {
  console.log('clear() clicked');
  collection.selections = [];
  Array.from(document.querySelectorAll('.selectedPolygon')).forEach(function(path) {
    path.classList.remove('selectedPolygon');
  })
  console.log('Data cleared');
}

// Send current selection back up-wire
function save() {
  console.log('save() clicked');
  const nameInput = element.find('#name');
  if (!nameInput.val() || collection.selections.length < 1) {
    alert('Collections must have a name and selected polygons');
  } else {
    collection.name = nameInput.val();
    console.log('You saved some data');
    console.log("Selection Name: ", collection.name);
    console.log(collection.selections);
  }
}

<IPython.core.display.Javascript object>

In [ ]:
%%javascript
require.undef('hello');

define('hello', ["@jupyter-widgets/base"], function(widgets) {

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        value_changed: function() {
            this.el.textContent = this.model.get('value');
        },
    });

    return {
        HelloView : HelloView
    };
});

In [ ]:
class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)

    def __init__(self):
        super().__init__()
        self.value = "Goodbye, World!"

In [ ]:
HelloWidget()

In [ ]:
h.value = "Goodbye, World!"

In [ ]:
h

In [ ]:
from shapely.geometry import MultiPolygon as MP, Polygon as P
from IPython.display import display
from ipywidgets.widgets import SelectMultiple
from ipykernel.comm import Comm

In [ ]:
%%javascript

Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target',
    function(comm, msg) {
        // comm is the frontend comm instance
        // msg is the comm_open message, which can carry data
        console.log(msg)
        // Register handlers for later messages:
        comm.on_msg(function(msg) {console.log(msg)});
        comm.on_close(function(msg) {console.log(msg)});
        comm.send({'foo': 0});
    });

In [ ]:
from ipykernel.comm import Comm

# Use comm to send a message from the kernel
my_comm = Comm(target_name='my_comm_target', data={})

# Add a callback for received messages.
@my_comm.on_msg
def _recv(msg):
    # Use msg['content']['data'] for the data in the message
    print(msg['content']['data'])

my_comm.send("HEELLLLLOOOOOOOOOOOOO!!!!!!!!!!!!!!")

In [6]:
class C(MP):
    def _repr_javascript_(self):
        w = SelectMultiple(options=list("abc"))
        display(w)
        svg = super()._repr_svg_()
        return '''
        element.html(`
        <style>
          #polygonGeometry path.selectedPolygon {
            fill: blue;
          }''' f'''
        </style>
        <button id="clearBtn">Clear</button>
        <input placeholder="Name this collection" id="name" />
        <button id="saveBtn">Save</button>
        <div id="polygonGeometry">{svg}</div>
        `)''' '''

        // Mutable collection data to send back up the wire "somehow"
        let collection = {
          name: '',
          selections: []
        }

        // Each path element is a polygon
        const polygons = element.find('#polygonGeometry path');

        // Add click event to polygons
        polygons.each(function(i, path) {
          path.addEventListener('click', function() {
            if (collection.selections.includes(i)) {
              path.classList.remove('selectedPolygon')
              remove(i);
              console.log('path #', i, ' removed');
            } else {
              path.classList.add('selectedPolygon')
              add(i);
              console.log('path #', i, ' added');
            }
          })
          console.log('path #', i, ' click set');
        })

        // Attach functions to buttons
        element.find('#clearBtn').click(clear);
        console.log('clearBtn action set');
        element.find('#saveBtn').click(save);
        console.log('saveBtn action set');

        // Add item to selection list
        function add(id) {
          collection.selections.push(id);
          console.log('pushed #', id);
        }

        // Remove item from selection list
        function remove(id) {
          collection.selections = collection.selections.filter(function(_id) {
            return _id !== id;
          })
          console.log('filtered #', id);
        }

        // Remove all items
        function clear() {
          console.log('clear() clicked');
          collection.selections = [];
          Array.from(document.querySelectorAll('.selectedPolygon')).forEach(function(path) {
            path.classList.remove('selectedPolygon');
          })
          console.log('Data cleared');
        }

        // Set collection name and send current selection back up-wire
        function save() {
          console.log('save() clicked');
          const nameInput = element.find('#name');
          if (!nameInput.val() || collection.selections.length < 1) {
            alert('Collections must have a name and selected polygons');
          } else {
            collection.name = nameInput.val();
            console.log('You saved some data');
            console.log("Selection Name: ", collection.name);
            console.log(collection.selections);
          }
        }
        '''

shapes = [
    [ [0,0], [1,0], [1,1] ],
    [ [0,0], [0,1], [1,1] ],
]

C([P(s) for s in shapes])

NameError: name 'MP' is not defined